# Getting Started with Python

## Import Statements

Python has a number of built in functionality, but many of the most useful tools and functions come require us to [import](https://docs.python.org/3/reference/simple_stmts.html#import) modules.

* Modules are pre-written blocks of code, sometimes referred to as packages or libraries.
* They are built for addressing specific tasks: e.g. linear algebra, mapping, plotting.

We're importing:
* [numpy](https://numpy.org/)
* [pandas](https://pandas.pydata.org/)
* [matplotlib](https://matplotlib.org/)

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
# This allows us to create interactive plots with matplotlib
%matplotlib notebook

# Step 2) Importing our data

* We'll use a Pandas to import three .csv files using the .read_csv() function to load them as "DataFrames"
    * We can set the header, automatically interpret dates, an set our table indexes

In [ ]:
# incident.csv contains postal code of the incident, province, municipality, along with the date and incident ID
Incident = pd.read_csv(
    'Data/incident.csv',
    delimiter = ',',
    header = 0,
    parse_dates=['date'],
    index_col=['id_incident']
)

# victim.csv contains information on the victms including age, race, etc.
Victim = pd.read_csv(
    'Data/victim.csv',
    delimiter = ',',
    header = 0, 
    index_col=['id_incident']
)

#police.csv contaisn information about the police department and oficer involved
Police = pd.read_csv(
    'Data/police.csv',
    delimiter = ',',
    header = 0, 
    index_col=['id_incident']
)
print('Data Loaded.')

# Step 3) Joining all our data

* The three files share the same index "id_incident"
    * We can use this unique identifier to combine all our files into one.
* Then we can set the date as the index.

In [ ]:
# First we'll join the incident and the victim records
Join_1 = Incident.join(Victim)
# Then we'll join the the police record
Join_2 = Join_1.join(Police)

# Resetting the index first allows us to keep 'id_incident'.
# We can call set_index('date') to set the date as the index.
# Setting drop to true, means that 'date' won't be duplicated as a column
PID_Canada = Join_2.reset_index().set_index('date',drop = True)

# Caling .columns function will give us all the column headers
print(PID_Canada.columns)

# Calling .head() will print the first 5 rows
PID_Canada.head()


# Step 4) Exploring the data

* Pandas has special functions
    * We can count, and summarize different variables

* We can quickly calculate statistics like averages or resample by a specific interval (e.g. Yearly) 
* We can use matplotlib.pyplot (plt) to display our information

In [ ]:
print('Number of Incidents: ',PID_Canada['id_incident'].count())
print('Descriptive Statistics Age: \n', PID_Canada['age'].describe())
print()

# Resample will aggregate the data over a given time interval
# 'Y' specifes we want years.  If you wanted monthly, you'd do 'M'
# .count() specifies how to aggregate.  If you have numeric data you can use .mean(), .std() etc. instead
# with text, you're limited to .count(), .first(), .last()
Yearly = PID_Canada.resample('Y').count()

## The linregress() function calcualtes a linear regression between the year and the number of killings
LR = stats.linregress(Yearly.index.year,Yearly['id_incident'])
print(LR)

# We can create a figure size 5x4
plt.figure(figsize=(5,4))

# .scatter() allows us to plot points
plt.scatter(Yearly.index.year,
            Yearly['id_incident'],
           color='black')

# .plots() allows us to plot lines
plt.plot(Yearly.index.year,
         Yearly.index.year*LR[0]+LR[1],
         color='red',
         label='Trendline: '+str(np.round(LR[0],2))+'\np = '+str(np.round(LR[3],3)))

# We can set some specifics here.
plt.title('Police Killings by Year in Canada')
plt.xlabel('Year')
plt.ylabel('Total')

# Calling .legend() will display all entites we set a label for
plt.legend()

# .tight_layout() allows us to make sure things fit nicely
plt.tight_layout()

### We can also aggregate using .groupby()

* This lets us conduct specific queries like:
    * ### What type of weapon did the victim have?

In [ ]:
# .groupby() accepts one or more records to aggregate by
# .count() tells us how to aggregate
Armed = PID_Canada.groupby(['armed_type']).count()

plt.figure(figsize=(7,4))

# .pie() creates a pie chart
plt.pie(
    Armed['id_victim'],
    labels=Armed.index,
    textprops={'fontsize': 8},
    autopct='%1.1f%%',
    wedgeprops={"edgecolor":"k",'linewidth': 1, 'linestyle': 'dashed'}
)
plt.title('Police Killings: Was the Victim Armed?')
plt.tight_layout()


### Lets make these categories easier to interpret
* We can create a dictionary define the replacements we want to make
* We can use some of pandas special functions to query and manipulate our data
    * The .loc[] function allows us to search for records
    * The .replace () function lets us replace values

In [ ]:
# Dictionaries use keys (eg. 'Vehicle') an values (eg. 'Other weapon')
# They let us quickly look up values by a key
replace_dict = {
    'Air gun, replica gun':'Other weapons',
    'Bat, club, other swinging object':'Other weapons',
    'Vehicle':'Other weapons',
    'Knife, axe, other cutting instruments':'Knife',
    'Unknown':'None'
          }

# We can loop through te keys in the dictionary and use them to replace the disired values
# .loc[] is a search command that allows us to perform specific querries
# we can use it in combination with an equal sign (=) to replace values for a given column(s)
for r in replace_dict.keys():
    PID_Canada.loc[PID_Canada['armed_type']==r,'armed_type']=replace_dict[r]
        
        
# Just making the same pic graph again
plt.figure(figsize=(6,4))
Armed = PID_Canada.groupby(['armed_type']).count()
plt.pie(
    Armed['id_victim'],
    labels=Armed.index,
    textprops={'fontsize': 8},
    autopct='%1.1f%%',
    wedgeprops={"edgecolor":"k",'linewidth': 1, 'linestyle': 'dashed'}
)
plt.title('Police Killings: Was the Victim Armed?')
plt.tight_layout()

### We can also make multiple plots together

In [ ]:
fig,ax= plt.subplots(2,1,figsize=(8,4))

# Groupby allows us to search fo multiple records
Force = PID_Canada.groupby(['Department','armed_type']).count()['id_victim']#.sort_values(ascending=True)

Force=Force.unstack()
col = Force.columns
# Force['Total']=Force.sum(axis=1)
# for c in col:
#     Force[c] /= Force['Total']

# Force = Force.sort_values(by='Total',ascending=False)#
# print(Force)
Force = Force.loc[Force['None']>1].sort_values(by='None')[-5:]

# Force = PID_Canada.groupby('Department').count()['id_victim'].sort_values(ascending=True)
ax[0].barh(Force.index,Force['None'],facecolor='#FF0000',edgecolor='black')
ax[0].set_title('Departments Killing the Most Unarmed People')


Charges = PID_Canada.loc[PID_Canada['armed_type']=='None'].groupby('Charges').count()
ax[1].pie(
    Charges['id_victim'],
    labels=Charges.index,
    textprops={'fontsize': 8},
    autopct='%1.1f%%',
    wedgeprops={"edgecolor":"k",'linewidth': 1, 'linestyle': 'dashed'}
)
ax[1].set_title('Was the Officer Charged for killing an unarmed person?')


plt.tight_layout()

### To look at the racial data, we need to normalize first
* Canada is predominately white, we have to scale each group by the size of their population to calculate a police killing rate
    * We want calculate the Police Killing Rate per Million Residents per Year for White, Black, and Indigenous people.
    * What should we use as the scale factor?
        * Hint the dataset spans the years 2000 to 2017
    

In [ ]:
Race=['Caucasian','Black','Indigenous']
Population=[25803368,1198545,1673780]

scale = 1e6/18

Count = PID_Canada.groupby('race')['id_incident'].count()

plt.figure()

i=0
for race,population in (zip(Race,Population)):
    if race == 'Total Population':
        rate=((Count.sum()/population)*scale)
    else:
        rate=((Count[race]/population)*1e6/18)
    plt.barh(i,rate,color='#FF0000',edgecolor='black')
    i += 1
plt.yticks([0,1,2],Race)
plt.title('Police Killing Rates by Race')
plt.xlabel('Killing/Year/Million People')
plt.tight_layout()
    
print(PID_Canada.index.year.unique().shape)#.count())

# Step 5 Saving Data

* We can save our data easily using the [.tocsv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) function



In [ ]:
PID_Canada.to_csv('Data/PID_Canada.csv')
print('Data Saved')
print(PID_Canada)